In [6]:
# default_exp core


In [2]:
# export

from peptide.imports import *
from peptide.data import ProteinDataset


In [3]:
# hide

from nbdev.showdoc import *


# Core
> Learner class and utility functions

In [3]:
# export


def visualize_2pcs(pcs, y):
    """Visualize 2 principal components."""
    fig, ax = plt.subplots(figsize=(10, 5))
    plot = plt.scatter(pcs[:, 0], pcs[:, 1], c=y, marker=".")
    ax.set_xlabel("PC 1")
    ax.set_ylabel("PC 2")
    ax.legend(handles=plot.legend_elements()[0], labels=["No", "Yes"])


In [4]:
# export


def visualize_3pcs(pcs, y):
    """Visualize 3 principal components."""
    fig, ax = plt.subplots(figsize=(10, 5))
    ax = fig.add_subplot(projection="3d")
    plot = ax.scatter(pcs[:, 0], pcs[:, 1], pcs[:, 2], c=y)
    ax.set_xlabel("PC 1")
    ax.set_ylabel("PC 2")
    ax.set_zlabel("PC 3")
    ax.legend(handles=plot.legend_elements()[0], labels=["No", "Yes"])


In [5]:
# export


def train_predict(X_train, y_train, X_test, y_test):
    """Utility helper function to quickly train and predict"""

    lr = LogisticRegression(max_iter=10000)
    lr.fit(X_train, y_train)

    svc = LinearSVC(max_iter=10000)
    svc.fit(X_train, y_train)

    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)

    lr_preds = lr.predict(X_test)
    svc_preds = svc.predict(X_test)
    xgb_preds = xgb.predict(X_test)

    scores = []
    for preds in [lr_preds, svc_preds, xgb_preds]:
        scores.append(
            [
                accuracy_score(y_test, preds),
                recall_score(y_test, preds),
                precision_score(y_test, preds),
                f1_score(y_test, preds),
            ]
        )

    return pd.DataFrame(
        scores, columns=["acc", "recall", "precision", "f1"], index=["lr", "svc", "xgb"]
    )


#hide

- To suppress warning in multithreaded runs - https://github.com/scikit-learn/scikit-learn/issues/12939
    - But makes it slow
- To revert - delete the following
```python
from sklearn.utils import parallel_backend
..
    with parallel_backend("multiprocessing"):

```

In [3]:
# export

# from sklearn.utils import parallel_backend


class Learner:
    """Class for training and prediction."""

    def __init__(
        self,
        dataset: ProteinDataset,
        ohe: bool = False,
        pca: bool = True,
        pca_n_components: int = 50,
    ):
        """Initialize learner for training and prediction."""
        self.classifiers = ["LogisticRegression", "LinearSVC", "XGBClassifier"]
        self.data = dataset
        self.pipeline = self.create_pipeline(
            self.data.X_train.shape[1], ohe, pca, pca_n_components
        )
        self.param_grids = self.create_param_grid()
        self.grid_list, self.train_results = [], []
        self.predict_results = None

    def create_pipeline(
        self, num_features: int, ohe: bool, pca: bool, pca_n_components
    ) -> Pipeline:
        """Create and return pipeline"""

        steps = []
        if ohe:
            steps.append(("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False)))
        if pca:
            steps.append(("pca", PCA(n_components=pca_n_components)))
        steps.append(("classifier", "passthrough"))

        pipe = Pipeline(steps)

        return pipe

    def create_param_grid(self) -> list:
        """Create and return a gird search param grid"""

        lr_grids = [
            {
                "classifier": [LogisticRegression()],
                "classifier__solver": ["lbfgs"],
                "classifier__penalty": ["l2"],
                "classifier__C": np.logspace(-2, 2, 5),  # default=1.0
                "classifier__max_iter": [1000, 5000, 10000],  # default=100
            },
            {
                "classifier": [LogisticRegression()],
                "classifier__solver": ["liblinear"],
                "classifier__penalty": ["l1", "l2"],
                "classifier__C": np.logspace(-2, 2, 5),
                "classifier__max_iter": [1000, 5000, 10000],
            },
        ]
        svm_grids = [
            {
                "classifier": [LinearSVC()],
                "classifier__loss": ["hinge"],  # default=squared_hinge
                "classifier__penalty": ["l2"],  # default=l2
                "classifier__C": np.logspace(-2, 2, 5),  # default=1.0
                "classifier__max_iter": [10000, 20000],  # default=1000
            },
            {
                "classifier": [LinearSVC()],
                "classifier__loss": ["squared_hinge"],
                "classifier__penalty": ["l1", "l2"],
                "classifier__C": np.logspace(-2, 2, 5),
                "classifier__max_iter": [10000, 20000],
            },
        ]
        xgb_grid = {
            "classifier": [XGBClassifier()],
            # "classifier__gamma": (5, 10),
            "classifier__learning_rate": np.linspace(0.03, 0.3, 4),  # default 0.1
            "classifier__max_depth": [3, 4, 5, 6],  # default 6
            "classifier__n_estimators": [100, 300],  # default 100
            # "classifier__subsample": (0.5, 1),
        }

        return [lr_grids, svm_grids, xgb_grid]

    def train(
        self, scoring: str = "accuracy", cv: int = 3, n_jobs: int = -1
    ) -> tuple[list, list]:
        """Run GridSearchCV for all models on X_train and y_train of dataset.

        Args:
            scoring: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
            cv: defaults to 3-fold cv
            n_jobs: defaults to -1 to use all cores
        Returns:
            result_list: list of grid search results
            grid_list: list of grid objects
        """

        result_list = []
        grid_list = []

        for classifier, param_grid in zip(self.classifiers, self.param_grids):
            print(f"Starting grid search for {classifier}")
            grid = GridSearchCV(
                estimator=self.pipeline,
                param_grid=param_grid,
                n_jobs=n_jobs,
                cv=cv,
                scoring=scoring,
                verbose=1,
            )

            # with parallel_backend("multiprocessing"):
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                grid.fit(self.data.X_train, self.data.y_train)

            result_list.append(pd.DataFrame.from_dict(grid.cv_results_))
            grid_list.append(grid)

        self.result_list = result_list
        self.grid_list = grid_list

        return self.result_list, self.grid_list

    def get_top_5_train_results(self) -> list:
        "Return top 5 results for each grid"
        results = []
        for result in self.result_list:
            results.append(result.sort_values("rank_test_score")[:5])
        return results

    def predict(self) -> pd.DataFrame:
        """Get predictions on the datasets X_test from best estimators of GridSearchCV."""
        results = []
        for classifier, grid in zip(self.classifiers, self.grid_list):
            preds = grid.predict(self.data.X_test)
            result = [
                classifier,
                grid.best_params_,
                accuracy_score(self.data.y_test, preds),
                recall_score(self.data.y_test, preds),
                precision_score(self.data.y_test, preds),
                f1_score(self.data.y_test, preds),
            ]
            results.append(result)

        self.predict_results = pd.DataFrame(
            results,
            columns=[
                "classifier",
                "best_params",
                "accuracy",
                "recall",
                "precision",
                "f1",
            ],
        )
        return self.predict_results


In [ ]:
# export


def visualize_elbow(X: np.ndarray, ks: list) -> None:
    """Visualize elbow plot for KMeans."""

    fig, ax = plt.subplots()
    inertias = []
    for k in ks:
        kmeans = KMeans(n_clusters=k, random_state=10).fit(X)
        inertias.append(kmeans.inertia_)
    plt.plot(ks, inertias)
    plt.xticks(ks)
    plt.xlabel("Number of clusters")
    plt.ylabel("Inertia")
    plt.title("Elbow plot")


In [5]:
# export


def plot_silhouette_scores(
    max_clusters: int, X: np.ndarray, random_state: int = 10
) -> None:
    """List and plot silhouette scores for KMeans."""

    silhouette_scores, clusters = [], []

    for n_clusters in range(2, max_clusters + 1):
        clusterer = KMeans(n_clusters=n_clusters, random_state=random_state)
        cluster_labels = clusterer.fit_predict(X)
        silhouette_avg = silhouette_score(X, cluster_labels)
        clusters.append(n_clusters)
        silhouette_scores.append(silhouette_avg)
        print(f"n_clusters: {n_clusters} -- avg silhouette score: {silhouette_avg}")

    # plot
    fig, ax = plt.subplots()
    plt.plot(clusters, silhouette_scores)
    plt.xticks(clusters)
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette scores")
    plt.title("Silhoutte Scores vs Clusters")


In [6]:
# export


def visualize_clusters(clust_lbls: np.ndarray, X: np.ndarray) -> None:
    """Visualize clusters in a plot of 2 principal components."""

    X_plot = pd.DataFrame(X[:, :2], columns=["pc1", "pc2"])
    X_plot["cluster_id"] = clust_lbls
    plt.figure(figsize=(12, 5))
    sns.scatterplot(
        data=X_plot,
        x=X_plot["pc1"],
        y=X_plot["pc2"],
        hue="cluster_id",
        palette="deep",
        legend=True,
    )


In [ ]:
# export


def pick_k(
    X_train: np.ndarray,
    X_test: np.ndarray,
    max_clusters: int = 10,
    ohe: bool = False,
    pca_n_components: int = 50,
) -> np.ndarray:
    """Visualize to help determine k for KMeans."""

    # concat X
    X = np.concatenate((X_train, X_test), axis=0)
    assert (X_train.shape[0] + X_test.shape[0]) == X.shape[0]
    # print(X_train.shape, X_test.shape, X.shape)

    if ohe:
        # One Hot Encode X
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
        X = ohe.fit_transform(X)
        # print(X.shape)

    # Dim Reduce X
    pca = PCA(n_components=pca_n_components)
    X_pca = pca.fit_transform(X)
    # print(X_pca.shape)

    # visualize elbow plot
    visualize_elbow(X_pca, np.arange(2, max_clusters))
    # visualize silhouette scores and plot
    plot_silhouette_scores(max_clusters=max_clusters, X=X_pca)

    return X_pca


In [ ]:
# export


def analyze_clusters(X: np.ndarray, k: int, random_state: int = 10) -> None:
    """Analyze clusters identified by KMeans."""

    km = KMeans(n_clusters=k, random_state=random_state).fit(X)
    print(f"Cluster counts: {Counter(km.labels_)}")
    visualize_clusters(km.labels_, X)


In [ ]:
# export


def run_label_spreading(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    ohe: bool = False,
    pca_n_components: int = 50,
) -> None:
    """Run Lanel Spreading and print classification report."""

    # concat X
    X = np.concatenate((X_train, X_test), axis=0)
    assert (X_train.shape[0] + X_test.shape[0]) == X.shape[0]
    # print(X_train.shape, X_test.shape, X.shape)

    # concat y
    y = np.concatenate((y_train, np.full(y_test.shape, -1)), axis=0)
    assert (y_train.shape[0] + y_test.shape[0]) == y.shape[0]
    # print(y_train.shape, y_test.shape, y.shape)

    if ohe:
        # One Hot Encode X
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
        X_ohe = ohe.fit_transform(X)
        # print(X.shape)

    # Dim Reduce X
    pca = PCA(n_components=pca_n_components)
    X_pca = pca.fit_transform(X)
    # print(X_pca.shape)

    # Run LableSpreading
    lbl_spread = LabelSpreading(kernel="knn", alpha=0.01)
    lbl_spread.fit(X_pca, y)
    semi_sup_preds = lbl_spread.transduction_[X_train.shape[0] :]
    assert semi_sup_preds.shape[0] == X_test.shape[0]

    # print result
    print(classification_report(y_test, semi_sup_preds))


## Export -

In [2]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_basics.ipynb.
Converted 01_data.ipynb.
Converted 02_core.ipynb.
Converted 03_onehot.ipynb.
Converted 04_lstm.ipynb.
Converted 05_transformer.ipynb.
Converted index.ipynb.
